<a href="https://colab.research.google.com/github/AmeliaAdhariani/capstoneproject/blob/main/REKOMENDASI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Input Data

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
#Ukuran filenya 25 GB ges dari kaggle
#upload file kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ayushparwal2026/country-population-from-1960-to-2022            Country Population from 1960 to 2022                75KB  2024-05-31 08:58:26           1120         27  0.9411765        
devarajv88/walmart-sales-dataset                                e-Commerce (Walmart) Sales Dataset                   5MB  2024-05-30 12:09:55           1410         24  1.0              
shreyanshverma27/online-sales-dataset-popular-marketplace-data  Online Sales Dataset - Popular Marketplace Data      7KB  2024-05-25 23:55:26           2321         39  1.0              
kanchana1990/vehicle-dataset-2024                               V

In [ ]:
! kaggle datasets download -d paramaggarwal/fashion-product-images-dataset

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset
License(s): copyright-authors
100% 23.1G/23.1G [05:25<00:00, 106MB/s] 
100% 23.1G/23.1G [05:25<00:00, 76.2MB/s]


In [ ]:
! unzip fashion-product-images-dataset.zip

Output streaming akan dipotong hingga 5000 baris terakhir.
  inflating: fashion-dataset/images/48352.jpg  
  inflating: fashion-dataset/images/48355.jpg  
  inflating: fashion-dataset/images/48357.jpg  
  inflating: fashion-dataset/images/48359.jpg  
  inflating: fashion-dataset/images/4836.jpg  
  inflating: fashion-dataset/images/48360.jpg  
  inflating: fashion-dataset/images/48361.jpg  
  inflating: fashion-dataset/images/48362.jpg  
  inflating: fashion-dataset/images/48363.jpg  
  inflating: fashion-dataset/images/48365.jpg  
  inflating: fashion-dataset/images/48367.jpg  
  inflating: fashion-dataset/images/48368.jpg  
  inflating: fashion-dataset/images/48369.jpg  
  inflating: fashion-dataset/images/4837.jpg  
  inflating: fashion-dataset/images/48370.jpg  
  inflating: fashion-dataset/images/48371.jpg  
  inflating: fashion-dataset/images/48372.jpg  
  inflating: fashion-dataset/images/48373.jpg  
  inflating: fashion-dataset/images/48374.jpg  
  inflating: fashion-dataset/im

In [ ]:
#librarynya blm lengkap ya masih pake yang sekiranya perlu dulu
#ntar tambahin aja
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import plotly.express as px
import zipfile
import tensorflow as tf
import urllib.request
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from mpl_toolkits.mplot3d import Axes3D
from tensorflow import keras
from sklearn.decomposition import PCA
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from os import listdir

### read and process data

In [ ]:
styles = pd.read_csv("/content/fashion-dataset/fashion-dataset/styles.csv", on_bad_lines='skip')
styles['filename'] = styles['id'].astype(str) + '.jpg'
styles = styles.dropna()
styles.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg


In [ ]:
images = pd.read_csv("/content/fashion-dataset/fashion-dataset/images.csv", on_bad_lines='skip')
images['id'] = images['filename'].apply(lambda x: x.replace(".jpg","")).astype(int)
images.head()

,filename,link,id
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,15970
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,39386
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,59263
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...,21379
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...,53759


In [ ]:
#ubah file nama agar sama dengan id
#baru di merge
images['id'] = images['filename'].apply(lambda x: x.replace(".jpg","")).astype(int)
images

,filename,link,id
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,15970
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,39386
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,59263
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...,21379
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...,53759
...,...,...,...
44441,17036.jpg,http://assets.myntassets.com/v1/images/style/p...,17036
44442,6461.jpg,http://assets.myntassets.com/v1/images/style/p...,6461
44443,18842.jpg,http://assets.myntassets.com/v1/images/style/p...,18842
44444,46694.jpg,http://assets.myntassets.com/v1/images/style/p...,46694


In [ ]:
#merge
merge_data = styles.merge(images, on='id', how='left')
print(merge_data.head())

      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName filename_x  \
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  15970.jpg   
1  2012.0  Casual             Peter England Men Party Blue Jeans  39386.jpg   
2  2016.0  Casual                       Titan Women Silver Watch  59263.jpg   
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  21379.jpg   
4  2012.0  Casual                          Puma Men Grey T-shirt  53759.jpg   

  filename_y                                            

In [ ]:
#Hapus yang tidak ada gambar
image_files = images['filename'].tolist()
merge_data['file_found'] = merge_data['filename_x'].apply(lambda x: x in image_files)
merge_data = merge_data[merge_data['file_found']].reset_index(drop=True)
merge_data.drop(columns=['file_found'], inplace=True)
print("Filtered Merged DataFrame:")
print(merge_data.head())

Filtered Merged DataFrame:
      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName filename_x  \
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  15970.jpg   
1  2012.0  Casual             Peter England Men Party Blue Jeans  39386.jpg   
2  2016.0  Casual                       Titan Women Silver Watch  59263.jpg   
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  21379.jpg   
4  2012.0  Casual                          Puma Men Grey T-shirt  53759.jpg   

  filename_y                 

In [ ]:
data = merge_data
data.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename_x,filename_y,link
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg,15970.jpg,http://assets.myntassets.com/v1/images/style/p...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg,39386.jpg,http://assets.myntassets.com/v1/images/style/p...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg,59263.jpg,http://assets.myntassets.com/v1/images/style/p...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg,21379.jpg,http://assets.myntassets.com/v1/images/style/p...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg,53759.jpg,http://assets.myntassets.com/v1/images/style/p...


In [ ]:
MAIN_DIR = '/tmp/fashion-dataset/'
IMG_SIZE = (224,224)
datagen = ImageDataGenerator(rescale=1/255.)
generator = datagen.flow_from_dataframe(dataframe=styles,
                                        directory=MAIN_DIR+'images',
                                        target_size=IMG_SIZE,
                                        x_col='filename',
                                        class_mode=None,
                                        batch_size=32,
                                        shuffle=False)

Found 44072 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 5 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
##MASIH ERROR
# Menentukan ukuran gambar
img_size = 224
# Menggunakan model InceptionV3
base_model = InceptionV3(include_top=False, input_shape=(img_size, img_size, 3))
# Membekukan semua layer pada base_model
for layer in base_model.layers:
    layer.trainable = False
# Membuat input layer
input_layer = Input(shape=(img_size, img_size, 3))
# Memasukkan input_layer ke base_model
x = base_model(input_layer)
# Menambahkan GlobalAveragePooling2D layer
output = GlobalAveragePooling2D()(x)
# Membuat model embeddings
embeddings = Model(inputs=input_layer, outputs=output)
# Menampilkan ringkasan model embeddings
embeddings.summary()
X = embeddings.predict(generator, verbose = 1)

87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
Total params: 21802784 (83.17 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________
1378/1378 [==============================] - 6423s 5s/step


In [ ]:
# PCA untuk reduksi dimensi
pca = PCA(2)
X_pca = pca.fit_transform(X)
styles[['pc1', 'pc2']] = X_pca

# Visualisasi hasil PCA
plt.figure(figsize=(20, 8))
sns.scatterplot(x='pc1', y='pc2', data=styles, hue='masterCategory')
plt.show()

NameError: name 'PCA' is not defined

In [ ]:
def read_img(image_path):
    image = load_img(os.path.join(path,image_path),target_size=(img_size,img_size,3))
    image = img_to_array(image)
    image = image/255.
    return image

In [ ]:
#ANN
import faiss

# Mengekstraksi fitur menggunakan InceptionV3
def extract_features(image_paths):
    base_model = InceptionV3(include_top=False, input_shape=(224, 224, 3))
    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs)
    outputs = GlobalAveragePooling2D()(x)
    model = Model(inputs, outputs)

    features = []
    for path in image_paths:
        img = load_img(path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        feature = model.predict(img_array)
        features.append(feature.flatten())

    return np.array(features)

# Baca file styles dan images
styles_df = pd.read_csv("/path/to/styles.csv", on_bad_lines='skip')
images_df = pd.read_csv("/path/to/images.csv", on_bad_lines='skip')

# Merge dataframes
merged_df = pd.merge(styles_df, images_df, on='id', how='inner')

# Ekstraksi fitur dari gambar
image_paths = ['/path/to/images/' + filename for filename in merged_df['filename']]
features = extract_features(image_paths)

# Membangun index FAISS
index = faiss.IndexFlatL2(features.shape[1])
index.add(features)

# Fungsi untuk mencari gambar yang mirip
def find_similar_images(query_feature, index, k=5):
    distances, indices = index.search(np.array([query_feature]), k)
    return indices[0]

# Memilih gambar secara acak
for _ in range(10):
    i = random.randint(0, len(styles_df) - 1)
    query_feature = features[i]
    similar_indices = find_similar_images(query_feature, index)

    # Menampilkan gambar input
    img1 = load_img(image_paths[i])
    plt.figure(figsize=(4, 4))
    plt.imshow(img1)
    plt.title("Input Image")
    plt.axis('off')

    # Menampilkan gambar produk yang mirip
    plt.figure(figsize=(20, 20))
    for j, idx in enumerate(similar_indices):
        img_similar = load_img(image_paths[idx])
        plt.subplot(1, 5, j + 1)
        plt.imshow(img_similar)
        plt.title(f"Similar Product #{j + 1}")
        plt.axis('off')

    plt.show()

In [ ]:
##Cara KNN
# y = styles_df['id']
# nearest_neighbours = KNeighborsClassifier(n_neighbors = 7)
# nearest_neighbours.fit(X,y)
# KNeighborsClassifier(n_neighbors=7)
# styles = styles.reset_index(drop=True)

# for _ in range(10):
#     i = random.randint(0,len(styles_df))
#     img1 = read_img(styles_df.loc[i,'filename'])
#     dist, index = nearest_neighbours.kneighbors(X=X[i,:].reshape(1,-1))
#     plt.figure(figsize = (4 , 4))
#     plt.imshow(img1)
#     plt.title("Input Image")
#     plt.axis('off')

#     plt.figure(figsize = (20 , 20))
#     for i in range(1,6):
#         plt.subplot(1 , 5, i)
#         plt.subplots_adjust(hspace = 0.5 , wspace = 0.3)
#         image = read_img(styles_df.loc[index[0][i],'filename'])
#         plt.imshow(image)
#         plt.title(f'Similar Product #{i}')
#         plt.axis('off')